In [91]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import os
import pandas as pd
import requests
from pprint import pprint
import time
import pymongo

# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.driver.maximize_window()
# URL of page to be scraped
url_base = "http://quotes.toscrape.com/"
next_page_link=''

quote_list=[]
while True:
    url=url_base+next_page_link
    browser.visit(url)
    html=browser.html
    quote_soup=BeautifulSoup(html,'html.parser')
    quotes=quote_soup.find_all('div',class_='quote')
    #loops each quote on the web page quotes.toscrape.com
    

    for quote in quotes:
        #finds the quote text
        q_text = quote.find('span',class_="text").text
        #print(q_text)
        #finds all tags 
        q_tags = quote.find_all('a',class_="tag")

        #list to store all tags
        tag_list=[]
        #loops through the quote tags and adds it to tag list
        for tag in q_tags:
            tag_text=tag.text
            tag_list.append(tag_text)
        
        #visiting author page
        #get author link
        try:
            #get link to author's page
            author_link = quote.find('a')['href']
            #print(author_link)
            #got to author's page
            browser.visit(url_base+author_link)
#           time.sleep(10)
            html2=browser.html
            #get the html soup from author_link
            author_soup=BeautifulSoup(html2,'html.parser')
            #get author_details from author_soup
            author_details=author_soup.find('div',class_='author-details')
            #get author's name from author_details
            author_name = author_details.find('h3').text
            #get author's birth date and place from author_details 
            author_born=author_details.find('span',class_='author-born-date').text + " "+ author_details.find('span',class_='author-born-location').text
            #get the description of the author
            author_description=author_details.find('div',class_="author-description").text
            #creates a dictionary for quote, tag, author's name, born, and description 
            quote_dict={"quote_text":q_text,"tags":tag_list,"author_name":author_name,"author_born":author_born,"author_description":author_description}
            #stores dictionary in a quote list
            quote_list.append(quote_dict)
            browser.back()
        except Exception as e:
            print("Scraping Author Page Complete")
            break
            
        
            
    try:
        next_page = quote_soup.find('li',class_='next')
        next_page_link = next_page.find('a')['href']
        #browser.close()
        #print(next_page_link)
    except Exception as e:
        print(f"Scraping Quotes Page {x+1} Complete")
        break
        #break

browser.quit()  
pprint(len(quote_list))


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Silho\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


Scraping Quotes Page 10 Complete
100


In [93]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
#creates a data base quotes_db
db=client.quotes_db
#creates a collection of quotes inside the quotes_db
collection = db.quotes
#insert quote_list as a MongoDB document
collection.insert_many(quote_list)